# Affordable

In [25]:
import os
import pandas as pd
import numpy as np

In [26]:
# read yearly income csv
Yearly_income=pd.read_csv('../data/curated/VIC_yearly_income_poa.csv')

In [27]:
# select the column we need
Yearly_income_poa = pd.DataFrame(Yearly_income, columns=["Postcode", "Median income in 2021-2022"])

In [28]:
# preprocess past rent data
External_rent= pd.read_excel('../data/raw/external_rent.xlsx',sheet_name='All Properties')
External_rent = External_rent.iloc[3:,[1,-1]]
External_rent = External_rent.dropna(axis='rows',how = 'all')

In [31]:
# rename column name
External_rent = External_rent.rename(columns={'Unnamed: 177':'Median rent price in 2021','Unnamed: 1':'LGA'})

In [32]:
External_rent

,LGA,Median rent price in 2021
3,Colac-Otway,335
4,Corangamite,315
5,Glenelg,330
6,Greater Geelong,400
7,Moyne,350
...,...,...
89,Group Total,400
90,Victoria,390
92,Metro,395
93,Non-Metro,350


In [33]:
# Use the weekly rent price to get the total annual rent price
External_rent['Median Yearly rent price in 2021'] = External_rent['Median rent price in 2021']*52

In [34]:
# Read the csv file of the converted poa and preprocess
postcode_LGA = pd.read_csv('../data/raw/postcode_info.csv')
attribute = postcode_LGA.loc[:,['Postcode','LGA Region']]
drop_dup = attribute.drop_duplicates(subset=['Postcode'])
rename = drop_dup.rename(columns={'LGA Region':'LGA'})

In [35]:
# merge two datasets by LGA
External_rent_poa = pd.DataFrame(pd.merge(rename,External_rent, on='LGA', how='inner'))
External_rent_poa.dropna(axis='rows')

,Postcode,LGA,Median rent price in 2021,Median Yearly rent price in 2021
0,3000,Melbourne,370,19240
1,3003,Melbourne,370,19240
2,3005,Melbourne,370,19240
3,3008,Melbourne,370,19240
4,3010,Melbourne,370,19240
...,...,...,...,...
714,3922,Bass Coast,350,18200
715,3923,Bass Coast,350,18200
716,3925,Bass Coast,350,18200
717,3991,Bass Coast,350,18200


In [36]:
# select the columns we need.
External_rent_poa = External_rent_poa.iloc[:,[0,3]]

In [37]:
# merge yearly income and yearly rent price by poa
rent_income = pd.DataFrame(pd.merge(External_rent_poa,Yearly_income_poa, on='Postcode', how='inner'))

In [38]:
# calculate affordable index by yearly income /yearly rent
rent_income['affordable index'] = rent_income['Median income in 2021-2022']/rent_income['Median Yearly rent price in 2021']

In [39]:
# save csv
rent_income.to_csv("../data/curated/rent_income21.csv")

In [40]:
# Sort by affordable index
top_index = rent_income.sort_values(by='affordable index', ascending=False)

In [41]:
# find the top 10 area with affordable index
top_index.iloc[:10,:]

,Postcode,Median Yearly rent price in 2021,Median income in 2021-2022,affordable index
405,3385,10400,60288.97,5.797016
414,3401,10400,60288.97,5.797016
415,3414,10400,56206.67,5.404488
416,3415,10400,56206.67,5.404488
417,3423,10400,56206.67,5.404488
418,3424,10400,56206.67,5.404488
412,3395,10400,53702.19,5.163672
410,3392,10400,53702.19,5.163672
408,3390,10400,53702.19,5.163672
411,3393,10400,53702.19,5.163672
